In [213]:
import os
from typing import Tuple, List

import yaml
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import netCDF4 as nc
import xarray as xr
import zarr

In [ ]:
def extract_time(configtime: str) -> tuple[list, list]:
    if '-' in configtime:
        trange = configtime.split('-')
        ranges = datetime.strptime(trange[0], '%Y%m')
        rangee = datetime.strptime(trange[1], '%Y%m')
        dd = (rangee - ranges).days + 1
        print(f'\n>>>> Extract Time Length --> {dd} <<<<')
        readday = [(ranges + timedelta(days=f)).strftime('%Y%m%d') for f in range(dd)]
        createmon = []
        curday = ranges
        while curday < rangee:
            createmon.append(curday.strftime('%Y%m'))
            curday = curday.replace(day=1) + timedelta(days=31)
            curday = curday.replace(day=1)       
        return  readday, createmon
    else:
        print(f'\n>>>> Extract Time Length --> 1 <<<<')
        readday = [datetime.strptime(configtime, '%Y%m%d').strftime('%Y%m%d')]
        
        return  readday, readday

def area_index(lon, lat, lonaxis, lataxis):
    lonaxes = np.where((lon >= min(lonaxis)) & (lon <= max(lonaxis)))[0]
    lataxes = np.where((lat >= min(lataxis)) & (lat <= max(lataxis)))[0]
    return lonaxes[0], lonaxes[-1], lataxes[0], lataxes[-1]

def NCwriter():
    savepath = config['savepath']
    savefile = savepath + create + '.nc'
    if os.path.exists(savefile):
        os.remove(savefile)

    ds = xr.Dataset(
        {
            "variable": (("time","var_num", "latitude", "longitude"), extract_data)
        },
        coords={
            "time": extract_data_time,
            "longitude": longitude[index_w: index_e + 1],
            "latitude": latitude[index_s: index_n + 1],
            "var_num": variable_name
        }
    )
    ds.to_netcdf(savefile)

In [ ]:
def Level_channel_name():
    level_channel_name = []
    if (not config['levelchannel'] == []) & (not config['level_variable'] == []):
        print('<- Level1 Channel: ', config['levelchannel'], ' ->')
        print('<- Level1 Variable: ', config['level_variable'], ' ->')
        for var in config['level_variable']:
            for le in config['levelchannel']:
                level_channel_name.append(var + str(le))

    if (not config['levelchannel2'] == []) & (not config['level_variable2'] == []):
        print('<- Level1 Channel: ', config['levelchannel2'], ' ->')
        print('<- Level1 Variable: ', config['level_variable2'], ' ->')
        for var in config['level_variable2']:
            for le in config['levelchannel2']:
                level_channel_name.append(var + str(le))

    if (not config['levelchannel3'] == []) & (not config['level_variable3'] == []):
        print('<- Level1 Channel: ', config['levelchannel3'], ' ->')
        print('<- Level1 Variable: ', config['level_variable3'], ' ->')
        for var in config['level_variable3']:
            for le in config['levelchannel3']:
                level_channel_name.append(var + str(le))
    
    return level_channel_name

In [222]:

def extract_data_single_var(data):
    """extract single variable by slice"""
    lonslic = slice(index_w, index_e + 1)
    latslic = slice(index_s, index_n + 1)
    timeslic = slice(config['timechannel'][0], config['timechannel'][-1] + 1)
    lonindex = config['single_variable_coor'].index('lon')
    latindex = config['single_variable_coor'].index('lat')
    timeindex = config['single_variable_coor'].index('time')
    singslic = [slice(None)] * 3
    singslic[lonindex] = lonslic
    singslic[latindex] = latslic
    singslic[timeindex] = timeslic
    return np.array(data[tuple(singslic)])

def extract_data_level_var(data, le):
    """extract level variable by slice and level channel"""
    lonslic = slice(index_w, index_e + 1)
    latslic = slice(index_s, index_n + 1)
    timeslic = slice(config['timechannel'][0], config['timechannel'][-1] + 1)
    levelslic = le
    lonindex = config['level_variable_coor'].index('lon')
    latindex = config['level_variable_coor'].index('lat')
    timeindex = config['level_variable_coor'].index('time')
    levelindex = config['level_variable_coor'].index('level')
    levslic = [slice(None)] * 4
    levslic[lonindex] = lonslic
    levslic[latindex] = latslic
    levslic[timeindex] = timeslic
    levslic[levelindex] = levelslic
    return np.array(data[tuple(levslic)])
    
    

In [223]:
yamlpath = './extract_test.yaml'
with open(yamlpath, 'r') as f:
    config = yaml.safe_load(f)

## set file name to read in hour
filename = '_12.nc'


In [224]:
print('========start extracting data========')
print('>>>> Config <<<<')
print(config)
print(f'\n>>>> Single Variable Num --> {len(config['single_variable'])} <<<<')
print(f'<- Single Variable: {config['single_variable']} ->')
print(f'\n>>>> Levles Variable Num --> {len(config['level_variable']) * len(config['levelchannel'])} <<<<')
print(f'<- Level Channel: {config['levelchannel']} ->')
print(f'<- Level Variable: {config['level_variable']} ->')
extime, creatfile = extract_time(config['time'])
variable_num = len(config['single_variable']) + len(config['level_variable']) * len(config['levelchannel'])
variable_name = config['single_variable']

if (not config['levelchannel'] ==[]) & (not config['level_variable'] == []):
    for var in config['level_variable']:
        for le in config['level']:
            lename = var + le
            variable_name.append(lename)

print(f'\n!!>> Variable Extract Num --> {variable_num} <<!!')
print(f'<- {variable_name} ->')

if config['path'][-1] == '/':
    filelist = [f for f in os.listdir(config['path']) if filename in f]
    filelist.sort()
    readfile = [(config['path'] + f + filename) for f in extime if (f + filename) in filelist]

else:
    filelist = config['path'].split('/')[-1]
    filelist.sort()
    readfile = [config['path']]
print(f'\n>>>> Extract File Number --> {len(readfile)} <<<<')
channels_name = config
if len(readfile) == 0:raise ValueError('Config time error, path number = 0 ')

========start extracting data========
>>>> Config <<<<
{'path': '/Users/zmy/dataset/', 'lon': 'longitude', 'lat': 'latitude', 'lonax': [70, 140], 'latax': [5, 60], 'time': '202409-202412', 'timechannel': [2, 9], 'single_variable': ['AOD550_DUST', 'AOD550_CLOUD', 'AOD550_BC'], 'single_variable_coor': ['time', 'lat', 'lon'], 'level_variable': ['u_m', 'v_m'], 'level_variable_coor': ['time', 'level', 'lat', 'lon'], 'levelchannel': [], 'savepath': './'}

>>>> Single Variable Num --> 3 <<<<
<- Single Variable: ['AOD550_DUST', 'AOD550_CLOUD', 'AOD550_BC'] ->

>>>> Levles Variable Num --> 0 <<<<
<- Level Channel: [] ->
<- Level Variable: ['u_m', 'v_m'] ->

>>>> Extract Time Length --> 92 <<<<

!!>> Variable Extract Num --> 3 <<!!
<- ['AOD550_DUST', 'AOD550_CLOUD', 'AOD550_BC'] ->

>>>> Extract File Number --> 2 <<<<


In [217]:
### extract variables


for create in creatfile:
    extract_data_time = []
    extract_data = []
    extractmon = [f for f in readfile if create in f]  
    if not extractmon: continue
    print(f'|> Files Exists in Create {create}: {len(extractmon)}')
    for num in range(len(extractmon)):
        print(f'|>|> Extract {num}/{len(extractmon)}', end='\r')
        extract_data_single = []
        extract_data_level = []
        fileread = extractmon[num]
        ncfile = xr.open_dataset(fileread, engine="netcdf4")
        filetime = datetime.strptime(filelist[num], '%Y%m%d_%H.nc')
        extracted_time = [filetime + timedelta(hours=8) + timedelta(hours=t *3) for t in range(config['timechannel'][0], config['timechannel'][-1] + 1)]
        extract_data_time += extracted_time
        ### find data index by lonax and latax in config
        
        if num == 0:
            longitude = ncfile['longitude']
            latitude = ncfile['latitude']
            lonax = config['lonax']
            latax = config['latax']
            index_w, index_e, index_s, index_n = area_index(longitude, latitude, lonax, latax)
        
        if config['single_variable']:
            for var in config['single_variable']:
                single_var = ncfile[var]    
                extract_data_single.append(extract_data_single_var(single_var))
    
        
        if (not config['levelchannel'] ==[]) & (not config['level_variable'] == []):
            for var in config['level_variable']:
                data = ncfile[var]
                for le in config['levelchannel']:
                    extract_data_level.append(extract_data_level_var(data, le))

        extract_data.append(np.array(extract_data_single + extract_data_level))
        
    if not extract_data: continue
    else:
        extract_data = np.concatenate(extract_data, axis=1).swapaxes(0, 1)
        NCwriter()

|> Files Exists in Create 202411: 2


NameError: name 'NCwriter' is not defined